# CNN Transfer&Train&Predict

# Init Config Setup

In [13]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00004"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

# get workflow version info
resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/')
data = json.loads(resp.json())

# get Active workflow version
wf_ver_id = 0
data = sorted(data, key=lambda k: k['pk'])
for config in data:
    if config["fields"]["active_flag"] == "Y":
        wf_ver_id = config["pk"]

wf_ver_id = str(wf_ver_id)
print("Network ID="+nn_id)
print("Workflow Version="+wf_ver_id)
print("")

def spaceprint(val, cnt):
    leng = len(str(val))
    cnt = cnt - leng
    restr = ""
    for i in range(cnt):
        restr += " "
    restr = restr+str(val)
    return restr

node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "param":{"traincnt": 1,
                                     "batch_size":10000,
                                     "predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                     "layeroutputs":32,
                                     "type":"category"
                                     }
                         ,"layer1": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                          ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                        })
netconf = json.loads(resp.json())
print("Network Config : {0}".format(netconf))
print("")

node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         ,"labels":[]

                     })
dataconf = json.loads(resp.json())
print("Train Data Config : {0}".format(dataconf))
print("")

node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("Eval Config : {0}".format(evalconf))
print("")

node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         ,"labels":[]

                     })
edataconf = json.loads(resp.json())
print("Eval Data Config : {0}".format(edataconf))
print("")

Network ID=nn00004
Workflow Version=10

Network Config : {'layer2': {'maxpoolstride': [2, 2], 'droprate': '0.8', 'maxpoolmatrix': [2, 2], 'type': 'cnn', 'layercnt': 1, 'padding': 'SAME', 'cnnfilter': [3, 3], 'cnnstride': [1, 1], 'active': 'relu'}, 'out': {'padding': 'SAME', 'node_out': 625, 'active': 'softmax'}, 'param': {'predictcnt': 10, 'traincnt': 1, 'batch_size': 10000}, 'modelpath': '/hoya_model_root/nn00004/10/netconf_node', 'modelname': 'model_nn00004_10', 'layer1': {'maxpoolstride': [2, 2], 'droprate': '0.8', 'maxpoolmatrix': [2, 2], 'type': 'cnn', 'layercnt': 2, 'padding': 'SAME', 'cnnfilter': [3, 3], 'cnnstride': [1, 1], 'active': 'relu'}, 'config': {'type': 'category', 'learnrate': 0.001, 'layeroutputs': 32, 'num_classes': 10}}

Train Data Config : {'preprocess': {'y_size': 32, 'x_size': 32, 'channel': 3}, 'source_path': '/hoya_src_root/nn00004/10/datasrc', 'store_path': '/hoya_str_root/nn00004/10/datasrc', 'labels': ['dog', 'cat', 'ca    t', 'motor', 'car', 'glove', 'airpl

# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

아래 API의 경우 Test Data인지 Train Data인지 선택할수 있는 Node 정보를 모두 가져와 준다.

In [24]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

get file node list
{'nn_wf_node_id': 'nn00004_2_datasrc', 'nn_wf_node_name': 'datasrc', 'nn_wf_node_desc': 'Train Data Node'}
{'nn_wf_node_id': 'nn00004_2_evaldata', 'nn_wf_node_name': 'evaldata', 'nn_wf_node_desc': 'Evaluation Data Node'}



아래 API는 한개의 파일을 Train Folder & Test Folder 에 넣어준다.

In [25]:
# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

create file node list
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/2/datasrc/sample_cnn_img.zip)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/2/datasrc/sample_cnn_img.zip)'}}
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/2/evaldata/sample_cnn_img.zip)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/2/evaldata/sample_cnn_img.zip)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br> 

In [16]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result":
        try:
            for tr in train["TrainResult"]:
                print(tr)
        except:
            maxcnt = 0
            line = ""
            for label in train["labels"]:
                if maxcnt<len(label):
                    maxcnt = len(label)

            for i in range(len(train["labels"])):
                for j in range(maxcnt+4):
                    line += "="

            label_sub = []
            for label in train["labels"]:
                label = spaceprint(label,maxcnt)
                label_sub.append(label)

            space = ""
            for s in range(maxcnt):
                space +=" "

            print(space, label_sub)
            print(space, line)
            for i in range(len(train["labels"])):
                predict_sub = []
                for j in range(len(train["predicts"][i])):
                    pred = spaceprint(train["predicts"][i][j],maxcnt)

                    predict_sub.append(pred)

                print(spaceprint(train["labels"][i],maxcnt), predict_sub)

['Trainning .................................................']
['Global Step:     1311, Training Batch Accuracy:  100.0%']

['Trainning .................................................']
['Global Step:     1312, Training Batch Accuracy:  95.95%']

         ['     dog', '     cat', ' ca    t', '   motor', '     car', '   glove', 'airplane', '    bolt']
     dog ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
     cat ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
 ca    t ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
   motor ['       0', '       0', '       0', '     145', '       0', '       2', '       0', '       3']
     car ['       0', '       0', '       0', '       2', '     146', '       1', '       0', '       1']
   glove ['       0', '       0', '       0', '       1', '       0', '     143', '       0', '       3']
airplane

# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [17]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')

FileName = 2motor.jpg
['motor', 'car', 'bolt', 'cat', 'airplane', 'glove', 'ca    t', 'dog']
[3.42963, 1.09599, 1.07893, -0.47218, -1.03896, -1.50153, -1.71871, -2.4775]

FileName = 2bolt.jpg
['bolt', 'airplane', 'glove', 'cat', 'motor', 'car', 'ca    t', 'dog']
[10.74874, 4.62584, 2.06559, -1.35931, -3.55601, -4.81932, -5.37274, -8.20261]

FileName = 1motor.jpg
['motor', 'bolt', 'car', 'airplane', 'glove', 'cat', 'ca    t', 'dog']
[10.17206, 3.69104, 1.93304, 1.9272, 1.47043, -3.12427, -6.1851, -6.47375]

FileName = 2glove.jpg
['glove', 'motor', 'bolt', 'dog', 'car', 'airplane', 'cat', 'ca    t']
[17.20678, 3.5709, 2.57347, -0.6, -1.32183, -2.21896, -3.23434, -4.89102]

FileName = 1bolt.jpg
['bolt', 'car', 'glove', 'motor', 'airplane', 'cat', 'ca    t', 'dog']
[19.97056, 4.01947, 2.99323, 2.86635, 2.61407, -8.42671, -13.5161, -17.85664]

FileName = 2air.jpg
['airplane', 'bolt', 'car', 'motor', 'glove', 'cat', 'ca    t', 'dog']
[9.8252, 3.99287, 0.22043, -1.38896, -2.12879, -3.0912, -3